In [1]:
import dask_geopandas as dgpd

# Use dask to read Parquet files in parallel
gdf_municipality = dgpd.read_parquet("data/NEW_municipalities.parquet").compute()
# gdf_protected = dgpd.read_parquet("data/protected_areas.parquet").compute()
# gdf_landcover = dgpd.read_parquet("data/land_cover.parquet").compute()
gdf_flood_5 = dgpd.read_parquet("data/flood_risk/FloodRisk_5yr.parquet").compute()
gdf_flood_25 = dgpd.read_parquet("data/flood_risk/FloodRisk_25yr.parquet").compute()
gdf_flood_100 = dgpd.read_parquet("data/flood_risk/FloodRisk_100yr.parquet").compute()

In [2]:
target_crs = "EPSG:32651"

# Convert GeoPandas to Dask GeoDataFrame
# ddf_landcover = dgpd.from_geopandas(gdf_landcover, npartitions=8)
# ddf_municipality = dgpd.from_geopandas(gdf_municipality, npartitions=8)
# ddf_protected = dgpd.from_geopandas(gdf_protected, npartitions=8)
ddf_flood_5 = dgpd.from_geopandas(gdf_flood_5, npartitions=8)
ddf_flood_25 = dgpd.from_geopandas(gdf_flood_25, npartitions=8)
ddf_flood_100 = dgpd.from_geopandas(gdf_flood_100, npartitions=8)

# Perform parallel CRS conversion
# ddf_landcover = ddf_landcover.to_crs(target_crs)
# ddf_municipality = ddf_municipality.to_crs(target_crs)
# ddf_protected = ddf_protected.to_crs(target_crs)
ddf_flood_5 = ddf_flood_5.to_crs(target_crs)
ddf_flood_25 = ddf_flood_25.to_crs(target_crs)
ddf_flood_100 = ddf_flood_100.to_crs(target_crs)

# Convert back to GeoPandas
# gdf_landcover = ddf_landcover.compute()
# gdf_municipality = ddf_municipality.compute()
# gdf_protected = ddf_protected.compute()
gdf_flood_5 = ddf_flood_5.compute()
gdf_flood_25 = ddf_flood_25.compute()
gdf_flood_100 = ddf_flood_100.compute()


## Compute Land Cover Areas per Municipality

In [ ]:
from joblib import Parallel, delayed
import geopandas as gpd
import pandas as pd
import numpy as np
import os

# Function to perform overlay on a subset of data
def overlay_partition(subset):
    return gpd.overlay(subset, gdf_municipality, how="intersection")

# Split landcover into smaller chunks
num_partitions = min(128, os.cpu_count() // 2)  # Adjust based on system load
gdf_split = np.array_split(gdf_landcover, num_partitions)

# Run overlay in parallel
results = Parallel(n_jobs=num_partitions)(delayed(overlay_partition)(gdf) for gdf in gdf_split)

# Combine results
landcover_joined = gpd.GeoDataFrame(pd.concat(results, ignore_index=True))

In [67]:
landcover_joined

,class_id,adm3_psgc,Count,Region,Province,Municipality,adm1_psgc,adm2_psgc,geo_level,len_crs,area_crs,len_km,area_km2,geometry
0,2,1.600303e+09,1,MIN,agusan del sur,esperanza,1.600000e+09,1.600300e+09,Mun,207067.0,1.331903e+09,207.0,1331.0,"POLYGON ((765559.913 954794.654, 765559.088 95..."
1,2,1.600303e+09,1,MIN,agusan del sur,esperanza,1.600000e+09,1.600300e+09,Mun,207067.0,1.331903e+09,207.0,1331.0,"POLYGON ((804058.776 954799.665, 804059.998 95..."
2,2,1.600303e+09,1,MIN,agusan del sur,esperanza,1.600000e+09,1.600300e+09,Mun,207067.0,1.331903e+09,207.0,1331.0,"POLYGON ((773414.998 954809.353, 773408.82 954..."
3,2,1.600303e+09,1,MIN,agusan del sur,esperanza,1.600000e+09,1.600300e+09,Mun,207067.0,1.331903e+09,207.0,1331.0,"POLYGON ((786359.216 954762.345, 786356.05 954..."
4,2,1.600303e+09,1,MIN,agusan del sur,esperanza,1.600000e+09,1.600300e+09,Mun,207067.0,1.331903e+09,207.0,1331.0,"POLYGON ((793679.752 954831.008, 793679.733 95..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627850,2,1.600303e+09,1,MIN,agusan del sur,esperanza,1.600000e+09,1.600300e+09,Mun,207067.0,1.331903e+09,207.0,1331.0,"POLYGON ((767548.065 954611.71, 767550.875 954..."
627851,2,1.600303e+09,1,MIN,agusan del sur,esperanza,1.600000e+09,1.600300e+09,Mun,207067.0,1.331903e+09,207.0,1331.0,"POLYGON ((791768.182 954750.004, 791764.37 954..."
627852,2,1.600303e+09,1,MIN,agusan del sur,esperanza,1.600000e+09,1.600300e+09,Mun,207067.0,1.331903e+09,207.0,1331.0,"POLYGON ((766692.937 954657.614, 766689.58 954..."
627853,2,1.600303e+09,1,MIN,agusan del sur,esperanza,1.600000e+09,1.600300e+09,Mun,207067.0,1.331903e+09,207.0,1331.0,"POLYGON ((805065.756 954618.501, 805064.426 95..."


In [68]:
import dask_geopandas as dask_gpd
import geopandas as gpd

# Convert GeoDataFrames to Dask format for parallel processing
ddf_landcover = dask_gpd.from_geopandas(landcover_joined, npartitions=16)  # Adjust partitions based on CPU cores
ddf_municipality = dask_gpd.from_geopandas(gdf_municipality, npartitions=16)

# Compute area in parallel
ddf_landcover["landcover_area"] = ddf_landcover.geometry.area
ddf_municipality["municipality_area"] = ddf_municipality.geometry.area

# Convert back to GeoPandas
landcover_joined = ddf_landcover.compute()
gdf_municipality = ddf_municipality.compute()

# Total area per municipality
municipality_area = gdf_municipality.set_index("adm3_psgc")["municipality_area"]

# Aggregate by municipality and land cover type
landcover_stats = (
    landcover_joined.groupby(["adm3_psgc", "class_id"])["landcover_area"]
    .sum()
    .unstack(fill_value=0)
    .div(municipality_area, axis=0)  # Normalize
).reset_index()

In [69]:
landcover_stats = landcover_stats.rename(
    columns={
        "adm3_psgc": "municipality_id",
        1: "% Forest",
        2: "% Crops",
        3: "% Flatlands",
        4: "% Built-up",
        5: "% Wetlands",
    }
)

landcover_stats

class_id,municipality_id,% Forest,% Crops,% Flatlands,% Built-up,% Wetlands
0,1.028030e+08,0.046231,0.421475,0.443372,0.059711,0.028706
1,1.028120e+08,0.000401,0.485153,0.316229,0.147883,0.049561
2,1.028150e+08,0.503109,0.137716,0.318179,0.030086,0.009873
3,1.028160e+08,0.001565,0.497563,0.376649,0.074523,0.049620
4,1.028230e+08,0.190629,0.094527,0.693880,0.008286,0.012678
...,...,...,...,...,...,...
737,1.908816e+09,0.000000,0.220524,0.001030,0.011617,0.766829
738,1.908818e+09,0.000000,0.798091,0.015941,0.047773,0.138195
739,1.908820e+09,0.000000,0.854234,0.006270,0.116677,0.022819
740,1.908821e+09,0.000000,0.750004,0.113976,0.035925,0.100095


## Compute % Protected Area per Municipality

In [ ]:
from joblib import Parallel, delayed
import geopandas as gpd
import pandas as pd
import numpy as np
import os

# Function to perform overlay on a subset of data
def overlay_partition(subset):
    return gpd.overlay(subset, gdf_municipality, how="intersection")

# Split landcover into smaller chunks
num_partitions = min(128, os.cpu_count() // 2)  # Adjust based on system load
gdf_split = np.array_split(gdf_protected, num_partitions)

# Run overlay in parallel
results = Parallel(n_jobs=num_partitions)(delayed(overlay_partition)(gdf) for gdf in gdf_split)

# Combine results
protected_joined = gpd.GeoDataFrame(pd.concat(results, ignore_index=True))

In [71]:
# Compute area in parallel
ddf_protected = dask_gpd.from_geopandas(protected_joined, npartitions=16)  # Adjust partitions based on CPU cores
ddf_protected["protected_area"] = ddf_protected.geometry.area
protected_joined = ddf_protected.compute()

municipality_area = gdf_municipality.set_index("adm3_psgc")["municipality_area"]

protected_stats = (
        protected_joined.groupby("adm3_psgc")["protected_area"]
        .sum()
        .div(municipality_area, axis=0)  # Normalize
        .fillna(0)  # Fill NaN values with 0
    ).reset_index()
protected_stats = protected_stats.rename(
    columns={
        "adm3_psgc": "Municipality ID",
        0: "% Protected Area",
    }
)

protected_stats

,Municipality ID,% Protected Area
0,1.028030e+08,0.001027
1,1.028120e+08,0.000000
2,1.028150e+08,0.180760
3,1.028160e+08,0.054449
4,1.028230e+08,0.005622
...,...,...
737,1.908816e+09,0.000000
738,1.908818e+09,0.000000
739,1.908820e+09,0.000000
740,1.908821e+09,0.000000


## Compute Flood Risk per Municipality

We add the Province Codes (adm2_psgc) to the flood risk gdf to reduce the overlay scope.

### Preprocess Data

In [14]:
gdf_municipality

,adm3_psgc,Count,Region,Province,Municipality,adm1_psgc,adm2_psgc,geo_level,len_crs,area_crs,len_km,area_km2,geometry
0,1.028030e+08,1,NLZ,ilocos norte,badoc,1.000000e+08,1.028000e+08,Mun,64985.0,80758428.0,64.0,80.0,"POLYGON ((232924.209 1989474.502, 232926.91 19..."
1,1.028120e+08,1,NLZ,ilocos norte,city of laoag,1.000000e+08,1.028000e+08,City,53964.0,110146974.0,53.0,110.0,"POLYGON ((248393.247 2016552.78, 248424.831 20..."
2,1.028150e+08,2,NLZ,ilocos norte,pagudpud,1.000000e+08,1.028000e+08,Mun,85639.0,193261277.0,85.0,193.0,"POLYGON ((285928.399 2055561.259, 285954.238 2..."
3,1.028160e+08,1,NLZ,ilocos norte,paoay,1.000000e+08,1.028000e+08,Mun,43881.0,69564806.0,43.0,69.0,"POLYGON ((241783.156 2006726.213, 241835.225 2..."
4,1.028230e+08,2,NLZ,ilocos norte,vintar,1.000000e+08,1.028000e+08,Mun,105732.0,531249117.0,105.0,531.0,"POLYGON ((275061.25 2039069.568, 279694.026 20..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,1.908816e+09,2,MIN,maguindanao del sur,pagalungan,1.900000e+09,1.908800e+09,Mun,127787.0,198328294.0,127.0,198.0,"MULTIPOLYGON (((689655.039 789220.539, 689656...."
738,1.908818e+09,1,MIN,maguindanao del sur,pandag,1.900000e+09,1.908800e+09,Mun,40384.0,44662780.0,40.0,44.0,"POLYGON ((706127.27 744160.271, 705163.117 743..."
739,1.908820e+09,1,MIN,maguindanao del sur,shariff aguak,1.900000e+09,1.908800e+09,Mun,43863.0,36575547.0,43.0,36.0,"POLYGON ((664236.134 759402.116, 664165.452 75..."
740,1.908821e+09,2,VIS,maguindanao del sur,shariff saydona mustapha,1.900000e+09,1.908800e+09,Mun,78042.0,55909439.0,78.0,55.0,"POLYGON ((671841.811 776638.023, 671920.415 77..."


In [ ]:
# Check for unique province count in gdf_municipality
print(gdf_municipality["Province"].nunique())
print(gdf_municipality["Province"].isna().sum())
print(gdf_municipality["adm2_psgc"].nunique())

# Display rows with missing province values
missing_province = gdf_municipality[gdf_municipality["Province"].isna()]
missing_province # HUCs

82
22
104


,adm3_psgc,Count,Region,Province,Municipality,adm1_psgc,adm2_psgc,geo_level,len_crs,area_crs,len_km,area_km2,geometry
153,3.301000e+08,12,NLZ,None,city of angeles,3.000000e+08,3.301000e+08,City,66451.0,1.211398e+08,66.0,121.0,"POLYGON ((244650.966 1679143.433, 244669.678 1..."
154,3.314000e+08,5,NLZ,None,city of olongapo,3.000000e+08,3.314000e+08,City,107151.0,1.412713e+08,107.0,141.0,"MULTIPOLYGON (((214421.225 1644596.896, 214393..."
220,4.312000e+08,5,SLZ,None,city of lucena,4.000000e+08,4.312000e+08,City,60969.0,8.357117e+07,60.0,83.0,"POLYGON ((352517.579 1546361.061, 353053.521 1..."
320,6.302000e+08,9,VIS,None,city of bacolod,6.000000e+08,6.302000e+08,City,93735.0,1.622480e+08,93.0,162.0,"POLYGON ((515237.851 1178348.844, 515154.242 1..."
321,6.310000e+08,8,VIS,None,city of iloilo,6.000000e+08,6.310000e+08,City,57927.0,7.263026e+07,57.0,72.0,"POLYGON ((455771.9 1189494.04, 455771.024 1189..."
402,7.306000e+08,19,VIS,None,city of cebu,7.000000e+08,7.306000e+08,City,88577.0,2.907363e+08,88.0,290.0,"MULTIPOLYGON (((597598.39 1159940.648, 597977...."
403,7.311000e+08,12,VIS,None,city of lapu-lapu,7.000000e+08,7.311000e+08,City,164331.0,6.729994e+07,164.0,67.0,"MULTIPOLYGON (((606153.03 1134354.281, 606154...."
404,7.313000e+08,5,VIS,None,city of mandaue,7.000000e+08,7.313000e+08,City,37143.0,3.110909e+07,37.0,31.0,"POLYGON ((605184.859 1145150.119, 605184.882 1..."
469,8.316000e+08,15,VIS,None,city of tacloban,8.000000e+08,8.316000e+08,City,101639.0,1.060181e+08,101.0,106.0,"MULTIPOLYGON (((714752.696 1255793.946, 714750..."
490,9.317000e+08,61,MIN,None,city of zamboanga,9.000000e+08,9.317000e+08,City,521492.0,1.496293e+09,521.0,1496.0,"MULTIPOLYGON (((395823.254 759783.209, 395838...."


In [11]:
gdf_flood_5.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   FloodRisk          207 non-null    float64 
 1   geometry           207 non-null    geometry
 2   Province           207 non-null    string  
 3   FloodReturnPeriod  207 non-null    string  
dtypes: float64(1), geometry(1), string(2)
memory usage: 9.2 KB


We need to:
- Manually determine the codes for the 15 missing values (5 province/s)
    - compostelavalley = davaodeoro
    - misamis oriental = misamisoriental
    - maguindanao: USE region code (now split into del norte and del sur) [1900000000.0]
    - metromanila: USE NCR region code [1300000000.0]
    - catanduanes: NO SITE INFO
- Also add region codes in case the codes don't match (Tacloban City has different code from Leyte)

Expected remaining missing value:
- adm2_psgc: 9
- adm1_psgc: 3 (catanduanes only)

In [3]:
gdf_flood_5['Province'] = gdf_flood_5['Province'].str.lower()

# Create a mapping of 'Province' to 'adm2_psgc' in gdf_municipality
# Get the first 'adm2_psgc' and 'adm1_psgc' for each 'Province' in gdf_municipality
province_to_codes = gdf_municipality.groupby('Province').agg({
    'adm2_psgc': 'first',  
    'adm1_psgc': 'first'   
})

# Remove spaces from the 'Province' values in province_to_code (index)
province_to_codes.index = province_to_codes.index.str.replace(" ", "")

# Manually define any special 'adm1_psgc' codes for specific provinces
manual_adm1_psgc = {
    'metromanila': 1300000000,
    'maguindanao': 1900000000,
}

# Change "misamis oriental to misamisoriental"
gdf_flood_5['Province'] = gdf_flood_5['Province'].str.replace("misamis oriental", "misamisoriental")

# Change "compostelavalley" to "davaodeoro"
gdf_flood_5['Province'] = gdf_flood_5['Province'].str.replace("compostelavalley", "davaodeoro")

# Map the province codes to gdf_flood_5
gdf_flood_5['adm2_psgc'] = gdf_flood_5['Province'].map(province_to_codes['adm2_psgc'])
gdf_flood_5['adm1_psgc'] = gdf_flood_5['Province'].map(province_to_codes['adm1_psgc'])

# Fill in the manually defined values for adm1_psgc (e.g., for 'metromanila')
gdf_flood_5['adm1_psgc'] = gdf_flood_5['adm1_psgc'].fillna(gdf_flood_5['Province'].map(manual_adm1_psgc))

# Display missing province values
print("Number of missing adm2 codes in gdf_flood_5:", gdf_flood_5["adm2_psgc"].isna().sum())
print("Number of missing adm1 codes in gdf_flood_5:", gdf_flood_5["adm1_psgc"].isna().sum())
missing_code = gdf_flood_5[gdf_flood_5["adm2_psgc"].isna() | gdf_flood_5["adm1_psgc"].isna()]
missing_code

Number of missing adm2 codes in gdf_flood_5: 9
Number of missing adm1 codes in gdf_flood_5: 3


,FloodRisk,geometry,Province,FloodReturnPeriod,adm2_psgc,adm1_psgc
84,1.0,"MULTIPOLYGON (((703425 730065, 703425 730035, ...",maguindanao,5yr,NaN,1.900000e+09
85,2.0,"MULTIPOLYGON (((702715.959 730064.752, 702712....",maguindanao,5yr,NaN,1.900000e+09
86,3.0,"MULTIPOLYGON (((697515 730425, 697515 730395, ...",maguindanao,5yr,NaN,1.900000e+09
87,1.0,"MULTIPOLYGON (((285082.91 1587802.12, 285072.9...",metromanila,5yr,NaN,1.300000e+09
88,2.0,"MULTIPOLYGON (((285072.91 1587802.12, 285072.9...",metromanila,5yr,NaN,1.300000e+09
89,3.0,"MULTIPOLYGON (((285062.91 1587802.12, 285062.9...",metromanila,5yr,NaN,1.300000e+09
147,1.0,"MULTIPOLYGON (((630030 1495910.469, 630000 149...",catanduanes,5yr,NaN,NaN
148,2.0,"MULTIPOLYGON (((630020 1495920, 630020 1495910...",catanduanes,5yr,NaN,NaN
149,3.0,"MULTIPOLYGON (((628810 1495970, 628810 1495960...",catanduanes,5yr,NaN,NaN


In [4]:
gdf_flood_25['Province'] = gdf_flood_25['Province'].str.lower()

# Create a mapping of 'Province' to 'adm2_psgc' in gdf_municipality
# Get the first 'adm2_psgc' and 'adm1_psgc' for each 'Province' in gdf_municipality
province_to_codes = gdf_municipality.groupby('Province').agg({
    'adm2_psgc': 'first',  
    'adm1_psgc': 'first'   
})

# Remove spaces from the 'Province' values in province_to_code (index)
province_to_codes.index = province_to_codes.index.str.replace(" ", "")

# Manually define any special 'adm1_psgc' codes for specific provinces
manual_adm1_psgc = {
    'metromanila': 1300000000,
    'maguindanao': 1900000000,
}

# Change "misamis oriental to misamisoriental"
gdf_flood_25['Province'] = gdf_flood_25['Province'].str.replace("misamis oriental", "misamisoriental")

# Change "compostelavalley" to "davaodeoro"
gdf_flood_25['Province'] = gdf_flood_25['Province'].str.replace("compostelavalley", "davaodeoro")

# Map the province codes to gdf_flood_25
gdf_flood_25['adm2_psgc'] = gdf_flood_25['Province'].map(province_to_codes['adm2_psgc'])
gdf_flood_25['adm1_psgc'] = gdf_flood_25['Province'].map(province_to_codes['adm1_psgc'])

# Fill in the manually defined values for adm1_psgc (e.g., for 'metromanila')
gdf_flood_25['adm1_psgc'] = gdf_flood_25['adm1_psgc'].fillna(gdf_flood_25['Province'].map(manual_adm1_psgc))

# Display missing province values
print("Number of missing adm2 codes in gdf_flood_25:", gdf_flood_25["adm2_psgc"].isna().sum())
print("Number of missing adm1 codes in gdf_flood_25:", gdf_flood_25["adm1_psgc"].isna().sum())
missing_code = gdf_flood_25[gdf_flood_25["adm2_psgc"].isna() | gdf_flood_25["adm1_psgc"].isna()]
missing_code

Number of missing adm2 codes in gdf_flood_25: 9
Number of missing adm1 codes in gdf_flood_25: 3


,FloodRisk,geometry,Province,FloodReturnPeriod,adm2_psgc,adm1_psgc
39,1.0,"MULTIPOLYGON (((630040 1495911.06, 630000 1495...",catanduanes,25yr,NaN,NaN
40,2.0,"MULTIPOLYGON (((629080 1495920, 629080 1495910...",catanduanes,25yr,NaN,NaN
41,3.0,"MULTIPOLYGON (((628810 1495970, 628810 1495960...",catanduanes,25yr,NaN,NaN
84,1.0,"MULTIPOLYGON (((703425 730035, 703425 730014.3...",maguindanao,25yr,NaN,1.900000e+09
85,2.0,"MULTIPOLYGON (((697455 730280.917, 697455 7302...",maguindanao,25yr,NaN,1.900000e+09
86,3.0,"MULTIPOLYGON (((703425 730125, 703425 730095, ...",maguindanao,25yr,NaN,1.900000e+09
93,1.0,"MULTIPOLYGON (((285132.91 1587822.12, 285132.9...",metromanila,25yr,NaN,1.300000e+09
94,2.0,"MULTIPOLYGON (((285082.91 1587812.12, 285082.9...",metromanila,25yr,NaN,1.300000e+09
95,3.0,"MULTIPOLYGON (((285072.91 1587802.12, 285072.9...",metromanila,25yr,NaN,1.300000e+09


In [5]:
gdf_flood_100['Province'] = gdf_flood_100['Province'].str.lower()

# Create a mapping of 'Province' to 'adm2_psgc' in gdf_municipality
# Get the first 'adm2_psgc' and 'adm1_psgc' for each 'Province' in gdf_municipality
province_to_codes = gdf_municipality.groupby('Province').agg({
    'adm2_psgc': 'first',  
    'adm1_psgc': 'first'   
})

# Remove spaces from the 'Province' values in province_to_code (index)
province_to_codes.index = province_to_codes.index.str.replace(" ", "")

# Manually define any special 'adm1_psgc' codes for specific provinces
manual_adm1_psgc = {
    'metromanila': 1300000000,
    'maguindanao': 1900000000,
}

# Change "misamis oriental to misamisoriental"
gdf_flood_100['Province'] = gdf_flood_100['Province'].str.replace("misamis oriental", "misamisoriental")

# Change "compostelavalley" to "davaodeoro"
gdf_flood_100['Province'] = gdf_flood_100['Province'].str.replace("compostelavalley", "davaodeoro")

# Map the province codes to gdf_flood_100
gdf_flood_100['adm2_psgc'] = gdf_flood_100['Province'].map(province_to_codes['adm2_psgc'])
gdf_flood_100['adm1_psgc'] = gdf_flood_100['Province'].map(province_to_codes['adm1_psgc'])

# Fill in the manually defined values for adm1_psgc (e.g., for 'metromanila')
gdf_flood_100['adm1_psgc'] = gdf_flood_100['adm1_psgc'].fillna(gdf_flood_100['Province'].map(manual_adm1_psgc))

# Display missing province values
print("Number of missing adm2 codes in gdf_flood_100:", gdf_flood_100["adm2_psgc"].isna().sum())
print("Number of missing adm1 codes in gdf_flood_100:", gdf_flood_100["adm1_psgc"].isna().sum())
missing_code = gdf_flood_100[gdf_flood_100["adm2_psgc"].isna() | gdf_flood_100["adm1_psgc"].isna()]
missing_code

Number of missing adm2 codes in gdf_flood_100: 9
Number of missing adm1 codes in gdf_flood_100: 3


,FloodRisk,geometry,Province,FloodReturnPeriod,adm2_psgc,adm1_psgc
36,1.0,"MULTIPOLYGON (((630020 1495910, 630020 1495909...",catanduanes,100yr,NaN,NaN
37,2.0,"MULTIPOLYGON (((630020 1495910, 630022.055 149...",catanduanes,100yr,NaN,NaN
38,3.0,"MULTIPOLYGON (((627680 1495960, 627680 1495950...",catanduanes,100yr,NaN,NaN
63,1.0,"MULTIPOLYGON (((702375 730365, 702375 730343.1...",maguindanao,100yr,NaN,1.900000e+09
64,2.0,"MULTIPOLYGON (((703425 730035, 703425 730014.3...",maguindanao,100yr,NaN,1.900000e+09
65,3.0,"MULTIPOLYGON (((703158.072 730162.337, 703155 ...",maguindanao,100yr,NaN,1.900000e+09
69,1.0,"MULTIPOLYGON (((285550 1590130, 285550 1590120...",metromanila,100yr,NaN,1.300000e+09
70,2.0,"MULTIPOLYGON (((285590 1590210, 285590 1590200...",metromanila,100yr,NaN,1.300000e+09
71,3.0,"MULTIPOLYGON (((287455 1590365, 287465 1590365...",metromanila,100yr,NaN,1.300000e+09


In [6]:
gdf_municipality_reduced = gdf_municipality[["adm3_psgc", "adm2_psgc","adm1_psgc","Municipality","geometry"]]
gdf_municipality_reduced

,adm3_psgc,adm2_psgc,adm1_psgc,Municipality,geometry
0,1.028030e+08,1.028000e+08,1.000000e+08,badoc,"POLYGON ((232924.209 1989474.502, 232926.91 19..."
1,1.028120e+08,1.028000e+08,1.000000e+08,city of laoag,"POLYGON ((248393.247 2016552.78, 248424.831 20..."
2,1.028150e+08,1.028000e+08,1.000000e+08,pagudpud,"POLYGON ((285928.399 2055561.259, 285954.238 2..."
3,1.028160e+08,1.028000e+08,1.000000e+08,paoay,"POLYGON ((241783.156 2006726.213, 241835.225 2..."
4,1.028230e+08,1.028000e+08,1.000000e+08,vintar,"POLYGON ((275061.25 2039069.568, 279694.026 20..."
...,...,...,...,...,...
737,1.908816e+09,1.908800e+09,1.900000e+09,pagalungan,"MULTIPOLYGON (((689655.039 789220.539, 689656...."
738,1.908818e+09,1.908800e+09,1.900000e+09,pandag,"POLYGON ((706127.27 744160.271, 705163.117 743..."
739,1.908820e+09,1.908800e+09,1.900000e+09,shariff aguak,"POLYGON ((664236.134 759402.116, 664165.452 75..."
740,1.908821e+09,1.908800e+09,1.900000e+09,shariff saydona mustapha,"POLYGON ((671841.811 776638.023, 671920.415 77..."


In [7]:
gdf_flood_5_reduced = gdf_flood_5[["adm1_psgc", "adm2_psgc","FloodRisk","geometry"]]
gdf_flood_25_reduced = gdf_flood_25[["adm1_psgc", "adm2_psgc","FloodRisk","geometry"]]
gdf_flood_100_reduced = gdf_flood_100[["adm1_psgc", "adm2_psgc","FloodRisk","geometry"]]

In [8]:
for gdf in [gdf_municipality, gdf_flood_5_reduced, gdf_flood_25_reduced, gdf_flood_100_reduced]:
    gdf.loc[:, 'adm2_psgc'] = gdf['adm2_psgc'].fillna(-1).astype(int)
    gdf.loc[:, 'adm1_psgc'] = gdf['adm1_psgc'].fillna(-1).astype(int)

In [ ]:
for gdf in [gdf_flood_5_reduced, gdf_flood_25_reduced, gdf_flood_100_reduced]:
    gdf["geometry"] = gdf["geometry"].simplify(tolerance=0.01, preserve_topology=True)
    gdf = gdf.explode(index_parts=False)

### Find intersections

In [ ]:
import geopandas as gpd
import pandas as pd

import logging
logging.basicConfig(level=logging.DEBUG)

def get_intersections(gdf_flood, gdf_municipality):
    municipality_processed = 0

    # Group flood data by (adm2_psgc, adm1_psgc) for faster lookup
    print("Grouping flood data by adm2_psgc and adm1_psgc...")
    flood_grouped = gdf_flood.groupby(['adm2_psgc', 'adm1_psgc'])

    intersections = []

    for _, municipality in gdf_municipality.iterrows():
        adm2_psgc = municipality['adm2_psgc']
        adm1_psgc = municipality['adm1_psgc']

        print(f"Processing Municipality: {municipality['Municipality']} (adm2_psgc: {adm2_psgc}, adm1_psgc: {adm1_psgc})")

        # Try to get matching flood data
        filtered_flood = None
        try:
            filtered_flood = flood_grouped.get_group((adm2_psgc, adm1_psgc))
        except KeyError:
            try:
                filtered_flood = flood_grouped.get_group((adm1_psgc,))
            except KeyError:
                print(f"No flood data found for {municipality['Municipality']}")
                continue  # Skip to next municipality

        # Convert single municipality row to GeoDataFrame
        municipality_gdf = gpd.GeoDataFrame([municipality], geometry='geometry', crs=gdf_flood.crs)

        # Perform intersection
        print("Calculating intersection...")
        intersection = gpd.overlay(filtered_flood, municipality_gdf, how='intersection')
        intersections.append(intersection)

        municipality_processed += 1
        print(f"Processed {municipality_processed}/{len(gdf_municipality)} municipalities.")

    # Combine all intersections into one GeoDataFrame
    if intersections:
        result = gpd.GeoDataFrame(pd.concat(intersections, ignore_index=True))
    else:
        result = gpd.GeoDataFrame()  # Return an empty GeoDataFrame if no intersections

    return result


intersections_5 = get_intersections(gdf_flood_5_reduced, gdf_municipality_reduced.iloc[:2])

Grouping flood data by adm2_psgc and adm1_psgc...
Processing Municipality: badoc (adm2_psgc: 102800000.0, adm1_psgc: 100000000.0)
Calculating intersection...


: 

: 

In [ ]:
import geopandas as gpd
import dask_geopandas as dgpd
import pandas as pd

import logging
logging.basicConfig(level=logging.DEBUG)


def get_intersections(gdf_flood, gdf_municipality):
    municipality_processed = 0

    # First, group flood data by adm2_psgc and adm1_psgc for efficient filtering
    print("Grouping flood data by adm2_psgc and adm1_psgc...")
    flood_grouped = gdf_flood.groupby(['adm2_psgc', 'adm1_psgc'])

    # Initialize an empty list to store the results of each intersection
    intersections = []

    # Convert the municipality DataFrame to Dask to enable parallel operations
    print("Converting municipality GeoDataFrame to Dask...")
    gdf_municipality = dgpd.from_geopandas(gdf_municipality, npartitions=4)

    def process_municipality(municipality):
        global municipality_processed
        # Get the current municipality's adm2_psgc and adm1_psgc
        adm2_psgc = municipality['adm2_psgc']
        adm1_psgc = municipality['adm1_psgc']

        print(f"Processing Municipality: {municipality['Municipality']} (adm2_psgc: {adm2_psgc}, adm1_psgc: {adm1_psgc})")

        # Try to get matching flood data
        filtered_flood = None
        try:
            filtered_flood = flood_grouped.get_group((adm2_psgc, adm1_psgc))
        except:
            try:
                filtered_flood = flood_grouped.get_group((adm1_psgc,))
            except:
                print(f"No flood data found for {municipality['Municipality']}")
                return None  # Skip to next municipality

        
        # Convert the municipality row to a GeoDataFrame
        print("Converting municipality row to GeoDataFrame...")
        municipality_gdf = gpd.GeoDataFrame([municipality], geometry='geometry', crs=gdf_flood.crs)

        # Perform the intersection
        print("Calculating intersection...")
        intersection = gpd.overlay(filtered_flood, municipality_gdf, how='intersection')

        municipality_processed += 1
        print(f"Processed {municipality_processed} municipalities!")
        return intersection

    # Apply the function to each municipality in parallel
    results = gdf_municipality.map_partitions(lambda df: [process_municipality(row) for _, row in df.iterrows()])

    # Flatten the list of results and remove None values
    intersections = [inter for sublist in results.compute() for inter in sublist if inter is not None]

    # Combine all the intersections into one GeoDataFrame
    if intersections:
        result = gpd.GeoDataFrame(pd.concat(intersections, ignore_index=True))
    else:
        result = gpd.GeoDataFrame()  # Empty GeoDataFrame if no intersections

    municipality_processed = 0
    return result


intersections_5 = get_intersections(gdf_flood_5_reduced, gdf_municipality_reduced)

Grouping flood data by adm2_psgc and adm1_psgc...
Converting municipality GeoDataFrame to Dask...
Processing Municipality: a (adm2_psgc: 1.0, adm1_psgc: 1.0)


ValueError: Metadata inference failed in `lambda`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
ValueError('must supply a same-length tuple to get_group with multiple grouping keys')

Traceback:
---------
  File "/data/students/ryan/anaconda3/envs/rapids-25.02/lib/python3.12/site-packages/dask/dataframe/utils.py", line 149, in raise_on_meta_error
    yield
  File "/data/students/ryan/anaconda3/envs/rapids-25.02/lib/python3.12/site-packages/dask/dataframe/dask_expr/_expr.py", line 4054, in emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3278644/3491018588.py", line 56, in <lambda>
    results = gdf_municipality.map

In [14]:
intersections_5.get_group((102800000, 100000000), None)

,adm1_psgc,adm2_psgc,FloodRisk,geometry
150,100000000.0,102800000.0,1.0,"MULTIPOLYGON (((251325 1972285, 251325 1972275..."
151,100000000.0,102800000.0,2.0,"MULTIPOLYGON (((251515 1972465, 251505 1972465..."
152,100000000.0,102800000.0,3.0,"MULTIPOLYGON (((251885 1972805, 251885 1972795..."


In [ ]:
# import geopandas as gpd

# def get_intersections(gdf_flood, gdf_municipality):
#     # Initialize an empty list to store the results of each intersection
#     intersections = []
#     municipality_processed = 0

#     # Iterate through each municipality in gdf_municipality
#     for _, municipality in gdf_municipality.iterrows():

#         # Get the current municipality's adm2_psgc
#         adm2_psgc = municipality['adm2_psgc']
#         adm1_psgc = municipality['adm1_psgc']

#         # Filter gdf_flood based on adm2_psgc first
#         filtered_flood = gdf_flood[gdf_flood['adm2_psgc'] == adm2_psgc]

#         # If no match on adm2_psgc, filter based on adm1_psgc
#         if filtered_flood.empty:
#             filtered_flood = gdf_flood[gdf_flood['adm1_psgc'] == adm1_psgc]
        
#         # If still no match, print a message and continue
#         if filtered_flood.empty:
#             print(f"No flood data found for {municipality['Municipality']}")
#             continue

#         # If there are rows left, perform the intersection
#         if not filtered_flood.empty:
#             # Convert the municipality row to a GeoDataFrame
#             municipality_gdf = gpd.GeoDataFrame([municipality], geometry='geometry', crs=gdf_flood.crs)

#             intersection = gpd.overlay(filtered_flood, municipality_gdf, how='intersection')
#             intersections.append(intersection)
        
#         municipality_processed += 1
#         print(f"Processed {municipality_processed}/{len(gdf_municipality)} municipalities.")

#         if municipality_processed == 5:
#             break

#     # Combine all the intersections into one GeoDataFrame (if there were any matches)
#     if intersections:
#         result = gpd.GeoDataFrame(pd.concat(intersections, ignore_index=True))
#     else:
#         result = gpd.GeoDataFrame()  # Empty GeoDataFrame if no intersections

#     return result

# intersections_5 = get_intersections(gdf_flood_5_reduced, gdf_municipality_reduced)

: 

: 

In [ ]:
print("Processing intersections for 25-year flood data...")
intersections_25 = get_intersections(gdf_flood_25_reduced, gdf_municipality_reduced)
print("Processing intersections for 100-year flood data...")
intersections_100 = get_intersections(gdf_flood_100_reduced, gdf_municipality_reduced)

### Sandbox Code

In [32]:
import geopandas as gpd
import concurrent.futures

municipality_processed = 0

def calculate_flood_percentage_per_municipality(municipality_gdf, flood_risk_gdf):
    """
    For each municipality, calculate the flood risk percentage (per flood level).
    Returns a dictionary with 'ID', '% level 1', '% level 2', '% level 3'.
    """
    global municipality_processed 

    flood_percentages = []

    # Iterate through each municipality and calculate the intersection with flood polygons
    for _, municipality in municipality_gdf.iterrows():
        # Perform the overlay (intersection) between municipality boundary and flood risk data
        intersection = gpd.overlay(flood_risk_gdf, municipality_gdf.iloc[[_]], how='intersection')

        # Calculate the total area of the municipality
        municipality_area = municipality.geometry.area
        
        # Count flood levels in the intersection
        flood_counts = intersection['FloodRisk'].value_counts()  # Assuming 'flood_risk' column has levels 1, 2, 3
        
        flood_percentages_for_municipality = {
            'ID': municipality['adm3_psgc'],
            'level_1': (flood_counts.get(1, 0) / municipality_area) * 100,
            'level_2': (flood_counts.get(2, 0) / municipality_area) * 100,
            'level_3': (flood_counts.get(3, 0) / municipality_area) * 100,
        }
        
        flood_percentages.append(flood_percentages_for_municipality)

        municipality_processed += 1
        print("Municipalities Processed: ", municipality_processed)
    
    return flood_percentages

def calculate_flood_percentage_parallel(municipality_gdf, flood_risk_gdf):
    """
    Parallelized version of calculating flood risk percentages for each municipality.
    Returns a GeoDataFrame with 'ID', '% level 1', '% level 2', '% level 3'.
    """
    global municipality_processed 
    
    def calculate_for_municipality(municipality_index):
        municipality = municipality_gdf.iloc[municipality_index:municipality_index+1]
        return calculate_flood_percentage_per_municipality(municipality, flood_risk_gdf)

    # Create a list of indices for municipalities
    municipality_indices = range(len(municipality_gdf))
    
    # Use concurrent futures for parallelization
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(calculate_for_municipality, municipality_indices))

    # Flatten the results and convert to DataFrame
    flat_results = [item for sublist in results for item in sublist]
    flood_percentages_df = pd.DataFrame(flat_results)

    # Create a GeoDataFrame with the result
    result_gdf = municipality_gdf[['geometry']].copy()  # Retain the geometry column
    result_gdf = result_gdf.join(flood_percentages_df.set_index('ID'), on='municipality_id')

    municipality_processed = 0

    return result_gdf

In [33]:
# municipality_gdf = gdf_municipality.sample(frac=0.01, random_state=42).reset_index(drop=True)
# flood_risk_gdf = gdf_flood_5.sample(frac=0.01, random_state=42).reset_index(drop=True)

# Parallelize the flood risk percentage calculation for all municipalities
flood_percentage_5_gdf = calculate_flood_percentage_parallel(gdf_municipality, gdf_flood_5)

flood_percentage_5_gdf

: 

: 

In [ ]:
# import geopandas as gpd

# def calculate_flood_percentage_per_municipality(municipality_gdf, flood_risk_gdf, test=True):
#     """
#     For each municipality, calculate the flood risk percentage (per flood level).
#     """
#     flood_percentages = []
#     municipality_ids = []

#     # Select only geometry and adm3_psgc columns from municipality_gdf
#     municipality_gdf = municipality_gdf[["adm3_psgc","geometry"]]

#     if test:
#         municipality_gdf = municipality_gdf.sample(frac=0.01, random_state=42).reset_index(drop=True)
#         flood_risk_gdf = flood_risk_gdf.sample(frac=0.01, random_state=42).reset_index(drop=True)


#     # Iterate through each municipality and calculate the intersection with flood polygons
#     for _, municipality in municipality_gdf.iterrows():
#         # Perform the overlay (intersection) between municipality boundary and flood risk data
#         intersection = gpd.overlay(flood_risk_gdf, municipality_gdf.iloc[[_]], how='intersection')

#         # Calculate the total area of the municipality
#         municipality_area = municipality.geometry.area
        
#         # Count flood levels in the intersection
#         flood_counts = intersection['flood_risk'].value_counts()  # Assuming 'flood_risk' column has levels 1, 2, 3
        
#         flood_percentages_for_municipality = {}
        
#         for flood_level in range(1, 4):  # Assuming flood levels are 1, 2, 3
#             flood_area = flood_counts.get(flood_level, 0)
#             flood_percentage = (flood_area / municipality_area) * 100
#             flood_percentages_for_municipality[flood_level] = flood_percentage
        
#         municipality_ids.append(id)
#         flood_percentages.append(flood_percentages_for_municipality)
    
#     return flood_percentages


In [3]:
import os
import torch

# No nvlink
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 

# Use a specific GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "4,2,3,5"

In [7]:
import cuspatial
import geopandas as gpd
import cudf
import numpy as np
import gc

# def extract_polygon_coordinates(gdf):
#     """Extracts polygon coordinates in a format compatible with cuspatial."""
#     polygon_x = []
#     polygon_y = []
#     ring_start_indices = [0]  # Start index for each polygon

#     for geom in gdf.geometry:
#         if geom.is_empty:
#             continue

#         # Extract exterior ring coordinates
#         x_coords, y_coords = geom.exterior.xy
#         polygon_x.extend(x_coords)
#         polygon_y.extend(y_coords)
#         ring_start_indices.append(len(polygon_x))  # New polygon starts here

#     return cudf.Series(polygon_x), cudf.Series(polygon_y), cudf.Series(ring_start_indices[:-1])


def rasterize_polygons_gpu(gdf, test=False, resolution=250):
    """
    Approximate rasterization by converting polygons into points and using GPU queries.
    """
    if test:
        gdf = gdf.sample(frac=0.01, random_state=42)

    # ✅ Explode MultiPolygons into separate Polygons
    
    print("Exploding polygons...")
    gdf = gdf.explode(index_parts=False).reset_index(drop=True)
    print("Converting to cuDF DataFrame...")
    gpu_dataframe = cuspatial.from_geopandas(gdf)

    # print("Calculating bounds...")
    # xmin, ymin, xmax, ymax = gdf.total_bounds
    # x_coords = np.arange(xmin, xmax, resolution)
    # y_coords = np.arange(ymin, ymax, resolution)

    # Generate a raster grid of points
    print("Generating grid of points...")
    x_points = np.linspace(gdf.total_bounds[0], gdf.total_bounds[2], num=resolution)
    y_points = np.linspace(gdf.total_bounds[1], gdf.total_bounds[3], num=resolution)
    xx, yy = np.meshgrid(x_points, y_points)

    # Convert to cuDF DataFrame and remove NaNs
    cdf_points = cudf.DataFrame({"x": xx.ravel(), "y": yy.ravel()}).dropna()

    # Ensure interleaving for cuSpatial
    print("Creating cuDF DataFrame for points...")
    xy = cdf_points.interleave_columns()
    cdf_points = cuspatial.GeoSeries.from_points_xy(xy)

    geometry = gpu_dataframe['geometry']


    # Process in batches of 30 polygons
    print("Performing point-in-polygon test...")

    batch_size = 30
    num_batches = len(geometry) // batch_size + (len(geometry) % batch_size > 0)

    results = []

    for i in range(num_batches):
        print(f"Processing batch {i + 1}/{num_batches}...")
        batch_polygons = geometry.iloc[i * batch_size : (i + 1) * batch_size]
        if len(batch_polygons) == 0:
            continue
        
        points_in_polygon = cuspatial.point_in_polygon(cdf_points, batch_polygons)
        results.append(points_in_polygon)

    # Merge all batch results into one DataFrame (each column corresponds to a batch of polygons)
    final_result = cudf.concat(results, axis=1)

    # Reduce across polygons (sum to count overlaps, or max to get a binary raster)
    raster_values = final_result.max(axis=1).to_numpy()  # Use max() for a binary mask

    # Reshape to match grid dimensions
    raster = raster_values.reshape(len(y_points), len(x_points))

    del final_result  # Delete variables no longer needed
    gc.collect()  # Force garbage collection
    
    return raster

In [9]:
import cupy as cp

def process_flood_risk(flood_gdf, muni_gdf, test=False):
    # Rasterize polygons on GPU
    print("Rasterizing flood polygons...")
    flood_raster_gpu = rasterize_polygons_gpu(flood_gdf, test=test)
    print("Rasterizing municipality polygons...")
    muni_raster_gpu = rasterize_polygons_gpu(muni_gdf, test=test)

    # Ensure flood_raster_gpu and muni_raster_gpu have the same shape BEFORE analysis
    if flood_raster_gpu.shape != muni_raster_gpu.shape:
        print("Flood and municipality rasters must have the same shape!")

    # Convert to cupy arrays (if they are not already)
    muni_raster_gpu = cp.asarray(muni_raster_gpu)  # Ensure it's a cupy array
    flood_raster_gpu = cp.asarray(flood_raster_gpu)  # Ensure it's a cupy array

    # Compute flood percentages without align()
    print("Computing flood percentages...")
    flood_percentages = cp.bincount(
        muni_raster_gpu.ravel(),  # Municipality categories
        weights=flood_raster_gpu.ravel()  # Flood intensities
    )

    return muni_raster_gpu, flood_raster_gpu, flood_percentages

# Process flood risk data
print("Processing 5-year flood risk data...")
muni_raster_gpu, flood_raster_gpu, flood_5_percentages = process_flood_risk(gdf_flood_5, gdf_municipality, test=True)

Processing 5-year flood risk data...
Rasterizing flood polygons...
Exploding polygons...
Converting to cuDF DataFrame...
Generating grid of points...
Creating cuDF DataFrame for points...
Performing point-in-polygon test...
Processing batch 1/1661...
Processing batch 2/1661...
Processing batch 3/1661...
Processing batch 4/1661...
Processing batch 5/1661...
Processing batch 6/1661...
Processing batch 7/1661...
Processing batch 8/1661...
Processing batch 9/1661...
Processing batch 10/1661...
Processing batch 11/1661...
Processing batch 12/1661...
Processing batch 13/1661...
Processing batch 14/1661...
Processing batch 15/1661...
Processing batch 16/1661...
Processing batch 17/1661...
Processing batch 18/1661...
Processing batch 19/1661...
Processing batch 20/1661...
Processing batch 21/1661...
Processing batch 22/1661...
Processing batch 23/1661...
Processing batch 24/1661...
Processing batch 25/1661...
Processing batch 26/1661...
Processing batch 27/1661...
Processing batch 28/1661...
P

In [14]:
muni_raster_gpu.shape

(250, 250)

In [ ]:
print("Processing 25-year flood risk data...")
flood_25_percentages = process_flood_risk(gdf_flood_25, gdf_municipality)
print("Processing 100-year flood risk data...")
flood_100_percentages = process_flood_risk(gdf_flood_100, gdf_municipality)

In [ ]:
# import cuspatial
# import cudf
# import geopandas as gpd
# from tqdm import tqdm
# import numpy as np

# def process_flood_risk(gdf_flood):
#     # Convert to cuDF and cuspatial format
#     gdf_flood_cudf = cudf.DataFrame(gdf_flood.drop(columns="geometry"))
#     gdf_flood_cudf["geometry"] = cuspatial.from_geopandas(gdf_flood.geometry)

#     # Split data into smaller chunks (to track progress)
#     # num_chunks = 1024  # Adjust based on GPU memory
#     # chunks = np.array_split(gdf_flood_cudf, num_chunks)

#     # Track progress
#     # results = []
#     # for i, chunk in enumerate(tqdm(chunks, desc="Processing GPU Intersection", unit="batch")):
#     #     result = cuspatial.point_in_polygon(chunk, gdf_municipality)
#     #     results.append(result)

#     # # Merge results
#     # final_result = cudf.concat(results)

#     # final_result = cuspatial.point_in_polygon(gdf_flood_cudf, gdf_municipality)
#     return final_result


# # Process flood risk data
# print("Processing 5-year flood risk data...")
# flood_risk_5 = process_flood_risk(gdf_flood_5)
# print("Processing 25-year flood risk data...")
# flood_risk_25 = process_flood_risk(gdf_flood_25)
# print("Processing 100-year flood risk data...")
# flood_risk_100 = process_flood_risk(gdf_flood_100)

# # Convert results back to GeoPandas
# flood_risk_5 = cuspatial.to_geopandas(flood_risk_5)
# flood_risk_25 = cuspatial.to_geopandas(flood_risk_25)
# flood_risk_100 = cuspatial.to_geopandas(flood_risk_100)



In [ ]:
# # Simplify polygons
# import geopandas as gpd
# import dask_geopandas as dgpd

# # Load GeoDataFrame
# def simplify_polygons(gdf, tolerance=0.001):
#     # Convert to Dask DataFrame
#     ddf = dgpd.from_geopandas(gdf, npartitions=512)  # More partitions for 256 cores

#     # Apply simplification in parallel using map_partitions
#     def simplify_partition(partition):
#         partition = partition.copy()  # Ensure it's a full copy
#         partition.loc[:, "geometry"] = partition["geometry"].simplify(tolerance, preserve_topology=True)
#         return partition
    
#     ddf = ddf.map_partitions(simplify_partition, meta=gdf)

#     # Compute back to GeoDataFrame
#     gdf_simplified = ddf.compute()

#     # Explode only if necessary
#     if gdf_simplified.geometry.geom_type.eq("MultiPolygon").any():
#         gdf_simplified = gdf_simplified.explode(index_parts=False)

#     # Reset index
#     return gdf_simplified.reset_index(drop=True)

# print("Processing 5-year flood risk data...")
# gdf_flood_5 = simplify_polygons(gdf_flood_5)
# print("Processing 25-year flood risk data...")
# gdf_flood_25 = simplify_polygons(gdf_flood_25)
# print("Processing 100-year flood risk data...")
# gdf_flood_100 = simplify_polygons(gdf_flood_100)

Processing 5-year flood risk data...


: 

: 

In [ ]:
# from joblib import Parallel, delayed
# from joblib_progress import joblib_progress

# import geopandas as gpd
# import numpy as np
# import os
# import warnings

# # Function to compute flood risk percentages per municipality
# def compute_flood_percentage(gdf_flood, return_period):
#     # Parallel overlay operation
#     num_partitions = min(128, os.cpu_count() // 2)  # Adjust based on available CPU cores
#     gdf_split = np.array_split(gdf_flood, num_partitions)
    
#     def overlay_partition(subset):
#         with warnings.catch_warnings():
#             warnings.simplefilter("ignore", category=FutureWarning)
#             return gpd.overlay(subset, gdf_municipality, how="intersection", keep_geom_type=False)

#     # Add joblib progress tracking
#     with joblib_progress(f"Computing flood intersection ({return_period}yr)", total=num_partitions):
#         flood_joined_parts = Parallel(n_jobs=num_partitions)(
#             delayed(overlay_partition)(gdf) for gdf in gdf_split
#         )

#     flood_joined = gpd.GeoDataFrame(pd.concat(flood_joined_parts, ignore_index=True))

#     # Compute flooded area per risk level in each municipality
#     flood_joined["flooded_area"] = flood_joined.geometry.area
#     flood_area_by_risk = (
#         flood_joined.groupby(["adm3_psgc", "FloodRisk"])["flooded_area"]
#         .sum()
#         .unstack(fill_value=0)
#     )

#     # Normalize by total municipality area
#     flood_percentage = (flood_area_by_risk.div(municipality_area, axis=0) * 100).fillna(0)
    
#     # Rename columns for clarity
#     flood_percentage = flood_percentage.reset_index()
#     flood_percentage.columns = ["municipality_id", f"%_low_risk_{return_period}yr", f"%_med_risk_{return_period}yr", f"%_high_risk_{return_period}yr"]
    
#     return flood_percentage

# # Compute percentage flood area for different return periods
# flood_5_percentage = compute_flood_percentage(gdf_flood_5, 5)


## Merge All Features

In [76]:
gdf_municipality = gdf_municipality.rename(
    columns={
        "adm3_psgc": "municipality_id",
    }
)

protected_stats = protected_stats.rename(
    columns={
        "Municipality ID": "municipality_id",
    }
)

In [77]:
# Merge all computed features into the municipality dataframe
final_features = gdf_municipality[["municipality_id"]].merge(landcover_stats, on="municipality_id", how="left")
final_features = final_features.merge(protected_stats, on="municipality_id", how="left")
# final_features = final_features.merge(flood_5_percentage, on="municipality_id", how="left")
# final_features = final_features.merge(flood_25_percentage, on="municipality_id", how="left")
# final_features = final_features.merge(flood_100_percentage, on="municipality_id", how="left")

final_features

,municipality_id,% Forest,% Crops,% Flatlands,% Built-up,% Wetlands,% Protected Area
0,1.028030e+08,0.046231,0.421475,0.443372,0.059711,0.028706,0.001027
1,1.028120e+08,0.000401,0.485153,0.316229,0.147883,0.049561,0.000000
2,1.028150e+08,0.503109,0.137716,0.318179,0.030086,0.009873,0.180760
3,1.028160e+08,0.001565,0.497563,0.376649,0.074523,0.049620,0.054449
4,1.028230e+08,0.190629,0.094527,0.693880,0.008286,0.012678,0.005622
...,...,...,...,...,...,...,...
737,1.908816e+09,0.000000,0.220524,0.001030,0.011617,0.766829,0.000000
738,1.908818e+09,0.000000,0.798091,0.015941,0.047773,0.138195,0.000000
739,1.908820e+09,0.000000,0.854234,0.006270,0.116677,0.022819,0.000000
740,1.908821e+09,0.000000,0.750004,0.113976,0.035925,0.100095,0.000000


In [79]:
final_features.to_parquet("model_features.parquet", index=False)